In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import datetime as dt

DATAPATH = './models/'
# GPU training
DEVICE = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {DEVICE}.")
torch.cuda.memory_allocated()

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training on device cuda.


0

In [2]:
from cocoBox import load_coco_dataset

BATCH_SIZE = 4
IMG_SIZE = 300
# Select image categories with which to train SSD
categories = ('horse', 'bird', 'cat', 'dog', 'person', 'car')
train_dataloader, valid_dataloader = load_coco_dataset(batch_size=BATCH_SIZE,
                                                       size=10000,
                                                       dim=IMG_SIZE,
                                                       cats=categories,
                                                       fetch_type='union')

loading annotations into memory...
Done (t=15.00s)
creating index...
index created!
Categories: {19: 0, 16: 1, 17: 2, 18: 3, 1: 4, 3: 5}
Dataset size: 10000
loading annotations into memory...
Done (t=14.19s)
creating index...
index created!
Categories: {19: 0, 16: 1, 17: 2, 18: 3, 1: 4, 3: 5}
Dataset size: 10000
Dataset with augmentation size: 20000


In [3]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, valid_loader):
    best_valid_loss = float('Inf')
    print('{} starting training'.format(dt.datetime.now()))
    for epoch in range(1, n_epochs + 1):
        train_loss = 0.0
        for images, targets in train_loader:
            images = torch.cat(list(map(lambda x: x.unsqueeze(0), images))).to(device=DEVICE)
            coords, conf = model(images)
            loss = loss_fn(coords, conf, targets['boxes'].to(device=DEVICE), targets['labels'].to(device=DEVICE))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        torch.cuda.empty_cache()
        # Validation
        with torch.no_grad():
            valid_loss = 0.0
            for images, targets in train_loader:
                images = images.to(device=DEVICE)
                coords, conf = model(images)
                loss = SSDloss(coords, conf, targets['boxes'].to(device=DEVICE), targets['labels'].to(device=DEVICE))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                valid_loss += loss.item()
        torch.cuda.empty_cache()
        # Verbose training
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Train {:.5f}, Valid {:.5f}'.format(dt.datetime.now(),
                                                                      epoch,
                                                                      train_loss,
                                                                      valid_loss))
        if valid_loss < best_valid_loss:
            torch.save(model.state_dict(), DATAPATH + 'ssd.pt')
            print(f'Saving {epoch}-th for {valid_loss = :2.5f}')
            best_valid_loss = valid_loss


In [4]:
import importlib
import SSD
importlib.reload(SSD)
import VGG16
importlib.reload(VGG16)
from SSD import SSD
import SSDLoss
importlib.reload(SSDLoss)
from SSDLoss import SSDLoss

ssd = SSD(class_num=len(categories)).to(device=DEVICE)
adam = torch.optim.Adam(ssd.parameters(), lr=1e-3)
training_loop(n_epochs=1000,
              optimizer=adam,
              loss_fn=SSDLoss().to(device=DEVICE),
              model=ssd,
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)

entered exception
2023-01-17 20:47:19.727884 starting training
Before base network: torch.Size([4, 3, 300, 300])
After base network: torch.Size([4, 512, 38, 38])


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches
# Load best models
ssd.load_state_dict(torch.load(DATAPATH + 'ssd.pt', map_location=DEVICE))
plt.figure(figsize=(12, 60))
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
mean = [-1 * x / y for x, y in zip(mean, std)]
std = [1 / x for x in std]
unnormalize = torchvision.transforms.Normalize(mean=mean, std=std)
for imgs, targets in valid_dataloader:
    if len(targets[0]['boxes']) < 2:
        continue
    fig, ax = plt.subplots()
    img = imgs[0]
    target = targets[0]
    ax.imshow(unnormalize(img).permute(1, 2, 0), vmin=0, vmax=1)
    # TODO: add labelling of rectangle box
    for idx, box in enumerate(target['boxes']):
        x = box[0] * img.shape[2]
        y = box[1] * img.shape[1]
        width = box[2] * img.shape[2]
        height = box[3] * img.shape[1]
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor="b", fill=False)
        ax.add_patch(rect)
        ax.text(x=x, y=y, s=categories[target['label'][idx]])
    offs, conf = ssd.predict(imgs, targets)
    # TODO: add labelling of rectangle box
    for idx, box in enumerate(offs):
        x = box[0] * img.shape[2]
        y = box[1] * img.shape[1]
        width = box[2] * img.shape[2]
        height = box[3] * img.shape[1]
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor="r", fill=False)
        ax.add_patch(rect)
        ax.text(x=x, y=y, s=categories[np.argmax(conf[idx])])
    plt.show()